In [1]:
!pip install --upgrade transformers datasets accelerate peft trl wandb python-dotenv

In [2]:
# This step is needed only on Apple Metal
!pip uninstall bitsandbytes -y

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv

dotenv_path = Path('../.env') # path to your .env file
load_dotenv(dotenv_path=dotenv_path)

True

Set wandb

In [4]:
import wandb

WANDB_KEY=os.getenv('WANDB_KEY')
wandb.login(key=WANDB_KEY)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ilya-ivensky (ilya-ivensky-free-lancer). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/ilyaivensky/.netrc


True

Set the model

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

#model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # Replace with the appropriate Llama 3.1 model name
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Replace with the appropriate Llama 3.1 model name

os.environ["WANDB_PROJECT"] = f"fine-tune-{model_name.replace('/', '-')}"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Set up LoRA configuration
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    lora_dropout=0,  # Dropout probability
    bias="none",  # Don't add bias to the LoRA adapters
    target_modules=['down_proj', 'gate_proj', 'o_proj', 'v_proj', 'up_proj', 'q_proj', 'k_proj'],
    task_type="CAUSAL_LM",
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

In [6]:
# Check if pad token exists, add it if missing
if tokenizer.pad_token is None:
   print(f'Added pad_token {tokenizer.eos_token}')
   tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"
tokenizer.is_split_into_words = True

# Update the model's token embeddings to accommodate the new pad token
model.resize_token_embeddings(len(tokenizer))

Added pad_token <|eot_id|>


Embedding(128256, 2048)

In [7]:
import torch
print(f'MPS available: {torch.backends.mps.is_available()}')
print(f'CUDA available: {torch.cuda.is_available()}')

device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

torch.manual_seed(42)
print(f'device={device}')

MPS available: True
CUDA available: False
device=mps


Prepare data

In [8]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split='all')
dataset

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [9]:
dataset[0]

{'Description': 'Q. What does abutment of the nerve root mean?',
 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?',
 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}

In [10]:
alpaca_propmt = """Below is an instruction that describes a task, paired with an input that provides further context. 

### Instruction: 
{}

### Input:
{}

### Response:
{}"""

def formatting_prompt_fn(example):

    instruction = example['Description'][3:] # Remove 'Q. '
    input = example['Patient']
    response = example['Doctor']

    text = alpaca_propmt.format(instruction, input, response) + tokenizer.eos_token

    return tokenizer(text)

formatted_dataset = dataset.map(formatting_prompt_fn, remove_columns=['Description', 'Patient', 'Doctor'], batched=False)
formatted_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 256916
})

In [11]:
train_test_split = formatted_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(20))

Training

In [12]:
import math

# Define the perplexity metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits and labels from NumPy arrays to PyTorch tensors
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    
    # Shift the labels so that they're aligned with the next token prediction
    labels = labels[:, 1:].reshape(-1)
    logits = logits[:, :-1].reshape(-1, logits.shape[-1])
    
    # Compute cross-entropy loss
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(logits, labels)

    # Compute perplexity from loss
    perplexity = math.exp(loss.item()) if loss.item() < 100 else float("inf")
    
    return {"perplexity": perplexity}

In [13]:
from torch.optim import AdamW
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./llama_lora_finetuned",
    eval_strategy="steps",
    eval_steps = 1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=128,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=1,
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    warmup_steps=10,
    load_best_model_at_end=True,
    report_to="wandb",
    seed=3407
)

optimizer = AdamW(
    model.parameters(), 
    lr=training_args.learning_rate, 
    weight_decay=training_args.weight_decay)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=small_eval_dataset,
    #peft_config=lora_config,
    #dataset_text_field="input_ids",
    tokenizer=tokenizer, 
    optimizers=(optimizer, None),  # No need for a scheduler here
    max_seq_length=tokenizer.model_max_length,
    compute_metrics=compute_metrics,
    packing=False,
    #dataset_kwargs={
    #    "add_special_tokens": False,
    #    "append_concat_token": False,
    #}
)


/opt/miniconda3/envs/torch-mlx/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/miniconda3/envs/torch-mlx/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/1605 [00:00<?, ?it/s]

{'loss': 3.4839, 'grad_norm': 0.6205085515975952, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 3.4462876319885254, 'eval_perplexity': 29.968461061195264, 'eval_runtime': 19.0327, 'eval_samples_per_second': 1.051, 'eval_steps_per_second': 1.051, 'epoch': 0.0}
{'loss': 3.4586, 'grad_norm': 0.6195924282073975, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


...

{'eval_loss': 3.2848517894744873, 'eval_perplexity': 25.76944214137618, 'eval_runtime': 50.1222, 'eval_samples_per_second': 0.399, 'eval_steps_per_second': 0.399, 'epoch': 0.02}
{'loss': 3.4172, 'grad_norm': 0.638789713382721, 'learning_rate': 9.86833855799373e-06, 'epoch': 0.02}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 3.277390718460083, 'eval_perplexity': 25.59080932714049, 'eval_runtime': 48.8885, 'eval_samples_per_second': 0.409, 'eval_steps_per_second': 0.409, 'epoch': 0.02}
{'loss': 3.3012, 'grad_norm': 0.5967254638671875, 'learning_rate': 9.862068965517241e-06, 'epoch': 0.02}


  0%|          | 0/20 [00:00<?, ?it/s]

{'eval_loss': 3.2700819969177246, 'eval_perplexity': 25.41783826593779, 'eval_runtime': 61.6283, 'eval_samples_per_second': 0.325, 'eval_steps_per_second': 0.325, 'epoch': 0.02}


In [ ]:
model.save_pretrained("./llama3_lora_model_finetuned")
tokenizer.save_pretrained("./llama3_lora_tokenizer_finetuned")

In [ ]:
wandb.finish()